In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [19]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [20]:
# data loader is used for automatic batching, sampling, shuffling and multiprocess data loading. 
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [22]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# to create a model in Pytorch our target class should extend from "nn.Module" class.
# and initialize the neural network layers in __init__. 
# Every nn.Module subclass implements the operations on input data in the forward method.
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            # --- Batch Norm example ---
            nn.BatchNorm1d(512), #<-- increase the accuracy
            #--------------------------
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model1 = NeuralNetwork().to(device)
print(model1)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): ReLU()
    (5): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [23]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model1.parameters(), lr=1e-3) 
#----- Optimizer with nesterov momentum ------
optimizer = torch.optim.SGD(model1.parameters(), lr=1e-3, momentum=0.9, nesterov=True)
#------------------------------------

In [24]:
# Training the model
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# Testing the model
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [25]:
# ------ Execute the model ----
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model1, loss_fn, optimizer)
    test(test_dataloader, model1, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.414472  [    0/60000]
loss: 0.737631  [ 6400/60000]
loss: 0.418127  [12800/60000]
loss: 0.626874  [19200/60000]
loss: 0.538404  [25600/60000]
loss: 0.476272  [32000/60000]
loss: 0.426128  [38400/60000]
loss: 0.573457  [44800/60000]
loss: 0.515640  [51200/60000]
loss: 0.430670  [57600/60000]
Test Error: 
 Accuracy: 84.5%, Avg loss: 0.432892 

Epoch 2
-------------------------------
loss: 0.326238  [    0/60000]
loss: 0.397537  [ 6400/60000]
loss: 0.267332  [12800/60000]
loss: 0.428600  [19200/60000]
loss: 0.367267  [25600/60000]
loss: 0.351664  [32000/60000]
loss: 0.335093  [38400/60000]
loss: 0.467820  [44800/60000]
loss: 0.390453  [51200/60000]
loss: 0.393347  [57600/60000]
Test Error: 
 Accuracy: 86.0%, Avg loss: 0.387322 

Epoch 3
-------------------------------
loss: 0.265562  [    0/60000]
loss: 0.334696  [ 6400/60000]
loss: 0.240960  [12800/60000]
loss: 0.355085  [19200/60000]
loss: 0.313560  [25600/60000]
loss: 0.309527  [32000/600